In [ ]:
#https://video.ittensive.com/python-advanced/data-44-structure-4.csv.gz
#https://video.ittensive.com/python-advanced/russia.json
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import descartes
#загружаем файл локально, так как  из его большого объёма, могут возникнуть сложности при работе
#будем использовать только колонки "Объект" и "Регион"
data = pd.read_csv("E:/ittensive/data-44-structure-4.csv", usecols=["Объект","Регион"])
#приводим регион к верхнему регистру, важно чтобы регистр "региона"
#  в обоих наборах данных был одинаковый 
data["Регион"] = data["Регион"].str.upper()
#выполняем группировку  по региону, посчитаем число объектов по каждому региону
data = data.groupby("Регион").count()
#выводим индекс оснвоного набора данных, чтобы просмотреть навзание регионов
#находим все расхождения, которые уже выведены в наборе нулевых значений
#в результирующем объединённом наборе данных
#print (data.index.values)
#загружаем гео-данные
geo = gpd.read_file("russia.json")
#приводим их к миркатеру
geo = geo.to_crs({"init": 'epsg:3857'})
#Название регионов в геоданных приводим к верхнему регистру
geo["NL_NAME_1"] = geo["NL_NAME_1"].str.upper()
#выполняем переманивание регионов в "гео-наборе", чтобы унифицировать их с основным набором данных
geo = geo.replace({
    "ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ": "ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА",
    "РЕСПУБЛИКА АДЫГЕЯ": "РЕСПУБЛИКА АДЫГЕЯ (АДЫГЕЯ)",
    "ЧУВАШСКАЯ РЕСПУБЛИКА": "ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ",
    "РЕСПУБЛИКА МАРИЙ-ЭЛ": "РЕСПУБЛИКА МАРИЙ ЭЛ",
    "РЕСПУБЛИКА СЕВЕРНАЯ ОСЕТИЯ": "РЕСПУБЛИКА СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ",
    "РЕСПУБЛИКА ТАТАРСТАН": "РЕСПУБЛИКА ТАТАРСТАН (ТАТАРСТАН)"
})
#осуществляем объединение наборов данных, (осинового и "гео")
#слияние выполняем по региону, за исходный набор данных возьмём "гео"
#в него вольём данные по объектам, чтобы вывести
#отображаем эти данные на картограмме
geo = pd.merge(left=geo, right=data,
              left_on="NL_NAME_1", right_on="Регион", how="left")
#чтобы найти проблемы по регионам, какие названия у нас расходятся
#выведем все строки где поля объект, значит число объектов в регионе "null"
#то есть  объединение корректно по этому региону не прошло (региона не было в исходном наборе)
#print (geo[geo["Объект"].isnull()])

#Создаём холст
fig = plt.figure(figsize=(16,9))
#добавляем туда область
area = plt.subplot(1, 1, 1)
#наносим область на картограмму, будем использовать количество объектов культурного наследия
# для числовой градации 
geo.plot(ax=area, legend=True, column="Объект", cmap="Reds")
#обрезаем картограмму, убираем Чукотку, и всё что западнее Калининграда
#задаём лимит по оси x, так как координаты указаны в десятках миллионов (1е7)
#поэтому нам нужен диапазон от 2 миллионов, до 20 миллионов
area.set_xlim(2e6, 2e7)
for _, region in geo.iterrows():
    #дополнительно наносим аннотацию, в виде числа объектов в каждом регионе 
    area.annotate(region["Объект"],
                #число объектов разместим в центре полигона, отвечающим за конкретный регио
                 xy=(region.geometry.centroid.x,
                    region.geometry.centroid.y), fontsize=8)
plt.show()
#выводим число объектов для Алтайского Края
print (geo[geo["NL_NAME_1"] == "АЛТАЙСКИЙ КРАЙ"]["Объект"])